# Web Scraping

In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests 
  

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_column', 100)

In [2]:
base_url = 'https://www.emlakjet.com'
filter_url = '/kiralik-konut/istanbul/?min_fiyat=9000'

In [3]:

page = requests.get(base_url + filter_url)

In [4]:
page.content

b'<!DOCTYPE html><html lang="tr"><head><link rel="shortcut icon" type="image/png" href="/static/img/favicon-green.png"/><style>\n    /* roboto-regular - latin_latin-ext */\n    @font-face {\n      font-family: \'Roboto\';\n      font-style: normal;\n      font-display: swap;\n      font-weight: 400;\n      src: url(\'/static/fonts/roboto-regular.eot\'); /* IE9 Compat Modes */\n      src: local(\'Roboto\'), local(\'Roboto-Regular\'),\n        url(\'/static/fonts/roboto-regular.eot?#iefix\') format(\'embedded-opentype\'), /* IE6-IE8 */\n        url(\'/static/fonts/roboto-regular.woff2\') format(\'woff2\'), /* Super Modern Browsers */\n        url(\'/static/fonts/roboto-regular.woff\') format(\'woff\'), /* Modern Browsers */\n        url(\'/static/fonts/roboto-regular.ttf\') format(\'truetype\'), /* Safari, Android, iOS */\n        url(\'/static/fonts/roboto-regular.svg#Roboto\') format(\'svg\'); /* Legacy iOS */\n    }\n    /* roboto-500 - latin_latin-ext */\n    @font-face {\n      font

In [5]:
soup = BeautifulSoup(page.content, 'html.parser')

In [6]:
paginations = soup.find('ul', {'class': '_49yNIg'}).find_all('li')[-2].text
paginations = int(paginations)
paginations

50

In [7]:
links = []

for i in range(1, paginations+1):
    page = requests.get(base_url + f'/kiralik-konut/istanbul/{i}/?min_fiyat=9000')
    soup = BeautifulSoup(page.content, 'html.parser')
    list = soup.find_all('a', {'class': '_3qUI9q'})

    for link in list:
        links.append(link.get('href')) 

In [8]:
len(links)

1500

In [9]:
page = requests.get(base_url + '/ilan/sancaktepe-meclis-de-hastane-ve-metroya-cok-yakin-23500-15789618/')
soup = BeautifulSoup(page.content, 'html.parser')

list = []
dict = {}

price_div = soup.find('div', {'class': '_2TxNQv'})
price = int(price_div.text.replace('TL', '').replace('.', '').replace(',', '').strip())
dict.update({'Fiyat': price})

location = soup.find('div', {'class': '_3VQ1JB'}).find('p').text
parts = location.split(' - ')
dict.update({'Şehir': parts[0]})
dict.update({'İlçe': parts[1]})
dict.update({'Mahalle': parts[2]})

average_div = soup.find('div', {'class': '_3tgtlK'})
if average_div is not None:
    average_price = average_div.select('p', {'class': '_1NQ0d5'})[1].text
    average_price = int(average_price.replace('TL', '').replace('.', '').replace(',', '').strip())
    dict.update({'Ortalama Fiyat': average_price})

features = soup.find_all('div', {'class': '_35T4WV'})
for feature in features:    
    divs = feature.find_all('div', class_='_1bVOdb')
    dict.update({divs[0].text: divs[1].text})


list.append(dict)
list

AttributeError: 'NoneType' object has no attribute 'text'

In [10]:
list = []
for link in links:
    page = requests.get(base_url + link)
    soup = BeautifulSoup(page.content, 'html.parser')
    
    dict = {}

    price_div = soup.find('div', {'class': '_2TxNQv'})

    if price_div is not None:
        price = price_div.text.replace('TL', '').replace('.', '').replace(',', '').strip()
        price = price.split('TL')[0] + 'TL'
        dict.update({'Fiyat': price})

    location = soup.find('div', {'class': '_3VQ1JB'})
    if location is not None:
        location = location.select('p')[0].text
        parts = location.split(' - ')
        dict.update({'Şehir': parts[0]})
        dict.update({'İlçe': parts[1]})
        dict.update({'Mahalle': parts[2]})

    average_div = soup.find('div', {'class': '_3tgtlK'})
    if average_div is not None:
        try:
            average_price = average_div.select('p', {'class': '_1NQ0d5'})[1].text
            average_price = int(average_price.replace('TL', '').replace('.', '').replace(',', '').strip())
            dict.update({'Ortalama Fiyat': average_price})
        except Exception as e:
            print(e)

    features = soup.find_all('div', {'class': '_35T4WV'})
    if features is not None:
        for feature in features:    
            divs = feature.find_all('div', class_='_1bVOdb')
            dict.update({divs[0].text: divs[1].text})


    list.append(dict)

ConnectionError: ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer'))

In [60]:
df = pd.DataFrame(list)
df = df.dropna(axis=1, how='all')
df.columns

Index(['Fiyat', 'Şehir', 'İlçe', 'Mahalle', 'Ortalama Fiyat', 'İlan Numarası',
       'İlan Güncelleme Tarihi', 'Kategorisi', 'Net Metrekare', 'Oda Sayısı',
       'Bulunduğu Kat', 'Isıtma Tipi', 'Depozito', 'Banyo Sayısı',
       'İlan Oluşturma Tarihi', 'Türü', 'Tipi', 'Brüt Metrekare',
       'Binanın Yaşı', 'Binanın Kat Sayısı', 'Kullanım Durumu',
       'Site İçerisinde', 'Fiyat Durumu', 'Yapı Durumu', 'Eşya Durumu',
       'Balkon Durumu', 'Yapı Tipi', 'WC Sayısı', 'Takas', 'Banyo Metrekare',
       'Salon Metrekare', 'WC Metrekare', 'Görüntülü Gezilebilir mi?', 'Aidat',
       'Balkon Tipi', 'Balkon Sayısı', 'Balkon Metrekare', 'Ä°lan NumarasÄ±',
       'Ä°lan GÃ¼ncelleme Tarihi', 'Oda SayÄ±sÄ±', 'BulunduÄŸu Kat',
       'IsÄ±tma Tipi', 'YapÄ± Durumu', 'EÅŸya Durumu', 'Site Ä°Ã§erisinde',
       'Banyo SayÄ±sÄ±', 'Ä°lan OluÅŸturma Tarihi', 'TÃ¼rÃ¼',
       'BrÃ¼t Metrekare', 'BinanÄ±n YaÅŸÄ±', 'BinanÄ±n Kat SayÄ±sÄ±',
       'KullanÄ±m Durumu', 'YapÄ± Tipi', 'WC SayÄ±sÄ±',
     

In [61]:
drop_columns = ['Ä°lan NumarasÄ±',
'Ä°lan GÃ¼ncelleme Tarihi', 'Oda SayÄ±sÄ±', 'BulunduÄŸu Kat',
'IsÄ±tma Tipi', 'YapÄ± Durumu', 'EÅŸya Durumu', 'Site Ä°Ã§erisinde',
'Banyo SayÄ±sÄ±', 'Ä°lan OluÅŸturma Tarihi', 'TÃ¼rÃ¼',
'BrÃ¼t Metrekare', 'BinanÄ±n YaÅŸÄ±', 'BinanÄ±n Kat SayÄ±sÄ±',
'KullanÄ±m Durumu', 'YapÄ± Tipi', 'WC SayÄ±sÄ±',
'GÃ¶rÃ¼ntÃ¼lÃ¼ Gezilebilir mi?', 'Balkon SayÄ±sÄ±']
df = df.drop(drop_columns, axis=1)

In [63]:
df.sample(10)

,Fiyat,Şehir,İlçe,Mahalle,Ortalama Fiyat,İlan Numarası,İlan Güncelleme Tarihi,Kategorisi,Net Metrekare,Oda Sayısı,Bulunduğu Kat,Isıtma Tipi,Depozito,Banyo Sayısı,İlan Oluşturma Tarihi,Türü,Tipi,Brüt Metrekare,Binanın Yaşı,Binanın Kat Sayısı,Kullanım Durumu,Site İçerisinde,Fiyat Durumu,Yapı Durumu,Eşya Durumu,Balkon Durumu,Yapı Tipi,WC Sayısı,Takas,Banyo Metrekare,Salon Metrekare,WC Metrekare,Görüntülü Gezilebilir mi?,Aidat,Balkon Tipi,Balkon Sayısı,Balkon Metrekare
963,10000TL,İstanbul,Avcılar,Denizköşkler Mahallesi,10170.0,15772696,26 Ağustos 2024,Kiralık,80 m²,2+1,Bahçe Katı,Isıtma Yok,10000 TL,1,26 Ağustos 2024,Konut,Daire,90 m²,21 Ve Üzeri,3,Boş,Hayır,Genel Fiyat,NaN,NaN,Var,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
700,10000TL,İstanbul,Avcılar,Denizköşkler Mahallesi,14690.0,15773824,26 Ağustos 2024,Kiralık,120 m²,3+1,Bahçe Katı,Kombi Doğalgaz,10000 TL,1,26 Ağustos 2024,Konut,Daire,130 m²,21 Ve Üzeri,5,Boş,Hayır,Genel Fiyat,NaN,NaN,Var,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Açık Balkon,1,NaN
812,22000TL,İstanbul,Büyükçekmece,Murat Çeşme Mahallesi,16055.0,15791612,29 Ağustos 2024,Kiralık,90 m²,2+1,3.Kat,Kombi Doğalgaz,22000 TL,1,29 Ağustos 2024,Konut,Daire,95 m²,0 (Yeni),4,Boş,Hayır,Genel Fiyat,Sıfır,Boş,Var,Betonarme,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
869,14000TL,İstanbul,Şişli,Kuştepe Mahallesi,7140.0,15781592,28 Ağustos 2024,Kiralık,25 m²,Stüdyo,1.Kat,Kombi Doğalgaz,NaN,1,28 Ağustos 2024,Konut,Daire,30 m²,1,4,Boş,Hayır,Genel Fiyat,NaN,Eşyalı,NaN,NaN,NaN,Yok,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
67,18000TL,İstanbul,Sancaktepe,Fatih Mahallesi,12730.0,15797637,30 Ağustos 2024,Kiralık,60 m²,2+1,1.Kat,Merkezi (Pay Ölçer),18000 TL,1,30 Ağustos 2024,Konut,Daire,67 m²,5-10,4,Boş,Hayır,Genel Fiyat,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,150 TL,NaN,NaN,NaN
875,20000TL,İstanbul,Şişli,Kuştepe Mahallesi,20230.0,15781583,28 Ağustos 2024,Kiralık,80 m²,2+1,1.Kat,Kombi Doğalgaz,NaN,1,28 Ağustos 2024,Konut,Daire,85 m²,4,4,Boş,Hayır,Genel Fiyat,NaN,Eşyalı,NaN,NaN,NaN,Yok,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
410,9750TL,İstanbul,Esenyurt,Mehmet Akif Ersoy Mahallesi,12600.0,15784806,28 Ağustos 2024,Kiralık,90 m²,2+1,Bahçe Katı,Kombi Doğalgaz,NaN,1,28 Ağustos 2024,Konut,Daire,100 m²,5-10,5,Boş,Hayır,Genel Fiyat,NaN,Boş,Var,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
756,42000TL,İstanbul,Üsküdar,Kandilli Mahallesi,46710.0,15796751,30 Ağustos 2024,Kiralık,78 m²,2+1,1.Kat,Kombi Doğalgaz,NaN,1,30 Ağustos 2024,Konut,Daire,90 m²,0 (Yeni),4,Boş,Hayır,Genel Fiyat,NaN,Boş,NaN,NaN,NaN,Yok,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
469,32500TL,İstanbul,Maltepe,Küçükyalı Mahallesi,28160.0,15782561,30 Ağustos 2024,Kiralık,100 m²,3+1,3.Kat,Kombi Doğalgaz,32500 TL,1,28 Ağustos 2024,Konut,Daire,110 m²,21 Ve Üzeri,4,Boş,Hayır,Genel Fiyat,NaN,NaN,Var,NaN,NaN,NaN,NaN,NaN,NaN,NaN,150 TL,NaN,NaN,NaN
736,24500TL,İstanbul,Kartal,Soğanlık Yeni Mahallesi,35250.0,15798714,30 Ağustos 2024,Kiralık,105 m²,3+1,1.Kat,Kombi Doğalgaz,3000 TL,1,30 Ağustos 2024,Konut,Daire,125 m²,21 Ve Üzeri,4,Boş,Hayır,Genel Fiyat,İkinci El,Boş,Var,Betonarme,2,Yok,NaN,NaN,NaN,Hayır,150 TL,Kapalı Balkon,1,NaN


In [64]:
df.columns

Index(['Fiyat', 'Şehir', 'İlçe', 'Mahalle', 'Ortalama Fiyat', 'İlan Numarası',
       'İlan Güncelleme Tarihi', 'Kategorisi', 'Net Metrekare', 'Oda Sayısı',
       'Bulunduğu Kat', 'Isıtma Tipi', 'Depozito', 'Banyo Sayısı',
       'İlan Oluşturma Tarihi', 'Türü', 'Tipi', 'Brüt Metrekare',
       'Binanın Yaşı', 'Binanın Kat Sayısı', 'Kullanım Durumu',
       'Site İçerisinde', 'Fiyat Durumu', 'Yapı Durumu', 'Eşya Durumu',
       'Balkon Durumu', 'Yapı Tipi', 'WC Sayısı', 'Takas', 'Banyo Metrekare',
       'Salon Metrekare', 'WC Metrekare', 'Görüntülü Gezilebilir mi?', 'Aidat',
       'Balkon Tipi', 'Balkon Sayısı', 'Balkon Metrekare'],
      dtype='object')

In [65]:
column_translation = {
    'Fiyat': 'Price',
    'Şehir': 'City',
    'İlçe': 'District',
    'Mahalle': 'Neighborhood',
    'İlan Numarası': 'Listing Number',
    'İlan Güncelleme Tarihi': 'Listing Update Date',
    'Kategorisi': 'Category',
    'Net Metrekare': 'Net Square Meters',
    'Oda Sayısı': 'Number of Rooms',
    'Bulunduğu Kat': 'Floor',
    'Isıtma Tipi': 'Heating Type',
    'Site İçerisinde': 'Within Site',
    'Fiyat Durumu': 'Price Status',
    'İlan Oluşturma Tarihi': 'Listing Creation Date',
    'Türü': 'Type',
    'Tipi': 'Kind',
    'Brüt Metrekare': 'Gross Square Meters',
    'Binanın Yaşı': 'Building Age',
    'Binanın Kat Sayısı': 'Number of Building Floors',
    'Kullanım Durumu': 'Usage Status',
    'Banyo Sayısı': 'Number of Bathrooms',
    'Yapı Durumu': 'Structure Status',
    'Eşya Durumu': 'Furnishing Status',
    'Depozito': 'Deposit',
    'Takas': 'Swap',
    'Banyo Metrekare': 'Bathroom Square Meters',
    'Salon Metrekare': 'Living Room Square Meters',
    'WC Metrekare': 'WC Square Meters',
    'Görüntülü Gezilebilir mi?': 'Virtual Tour Available?',
    'Yapı Tipi': 'Building Type',
    'Aidat': 'Maintenance Fee',
    'Balkon Durumu': 'Balcony Status',
    'WC Sayısı': 'Number of WCs',
    'Balkon Tipi': 'Balcony Type',
    'Balkon Sayısı': 'Number of Balconies',
    'Balkon Metrekare': 'Balcony Square Meters',
    'Ortalama Fiyat': 'Average Price'
}


df.rename(columns=column_translation, inplace=True)

In [66]:
df.sample(10)

,Price,City,District,Neighborhood,Average Price,Listing Number,Listing Update Date,Category,Net Square Meters,Number of Rooms,Floor,Heating Type,Deposit,Number of Bathrooms,Listing Creation Date,Type,Kind,Gross Square Meters,Building Age,Number of Building Floors,Usage Status,Within Site,Price Status,Structure Status,Furnishing Status,Balcony Status,Building Type,Number of WCs,Swap,Bathroom Square Meters,Living Room Square Meters,WC Square Meters,Virtual Tour Available?,Maintenance Fee,Balcony Type,Number of Balconies,Balcony Square Meters
1072,22000arrow_downward%12TL,İstanbul,Şişli,Kuştepe Mahallesi,14280.0,15769772,26 Ağustos 2024,Kiralık,55 m²,1+1,4.Kat,Kombi Doğalgaz,NaN,1,26 Ağustos 2024,Konut,Daire,60 m²,5-10,4,Boş,Hayır,Genel Fiyat,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1140,15000TL,İstanbul,Bahçelievler,Cumhuriyet Mahallesi,13140.0,15767210,25 Ağustos 2024,Kiralık,85 m²,2+1,4.Kat,Kombi Doğalgaz,30000 TL,1,25 Ağustos 2024,Konut,Daire,90 m²,21 Ve Üzeri,5,Boş,Hayır,Genel Fiyat,İkinci El,Boş,Var,Betonarme,1,Yok,NaN,NaN,NaN,Evet,NaN,NaN,1,NaN
166,17500TL,İstanbul,Sarıyer,Büyükdere Mahallesi,18760.0,15794698,30 Ağustos 2024,Kiralık,60 m²,1+1,Bahçe Katı,Kombi Doğalgaz,NaN,1,30 Ağustos 2024,Konut,Daire,70 m²,21 Ve Üzeri,3,Boş,Hayır,Genel Fiyat,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
928,15500TL,İstanbul,Bağcılar,Bağlar Mahallesi,10458.0,15774265,27 Ağustos 2024,Kiralık,28 m²,Stüdyo,7.Kat,Merkezi (Pay Ölçer),NaN,1,27 Ağustos 2024,Konut,Daire,42 m²,5-10,18,Boş,Evet,Genel Fiyat,NaN,Boş,NaN,NaN,NaN,Yok,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1146,20000TL,İstanbul,Pendik,Batı Mahallesi,15680.0,15767113,25 Ağustos 2024,Kiralık,75 m²,2.5+1,3.Kat,Kombi Doğalgaz,NaN,1,25 Ağustos 2024,Konut,Daire,80 m²,21 Ve Üzeri,4,Kiracı Oturuyor,Hayır,Genel Fiyat,NaN,Boş,NaN,NaN,NaN,Yok,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1093,18000TL,İstanbul,Eyüpsultan,Nişancı Mahallesi,11580.0,15768488,26 Ağustos 2024,Kiralık,50 m²,1+1,2.Kat,Kombi Doğalgaz,NaN,1,26 Ağustos 2024,Konut,Daire,60 m²,16-20,3,Boş,Hayır,Genel Fiyat,NaN,Eşyalı,NaN,NaN,NaN,Yok,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
415,10000TL,İstanbul,Esenyurt,Mehmet Akif Ersoy Mahallesi,7560.0,15784684,28 Ağustos 2024,Kiralık,50 m²,1+1,4.Kat,Kombi Doğalgaz,NaN,1,28 Ağustos 2024,Konut,Daire,60 m²,11-15,5,Boş,Hayır,Genel Fiyat,NaN,Boş,NaN,NaN,NaN,Yok,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
457,18500TL,İstanbul,Maltepe,Zümrütevler Mahallesi,21165.0,15783230,29 Ağustos 2024,Kiralık,75 m²,2+1,4.Kat,Kombi Doğalgaz,20000 TL,1,28 Ağustos 2024,Konut,Daire,85 m²,21 Ve Üzeri,5,Boş,Hayır,Genel Fiyat,İkinci El,Eşyalı,Var,Betonarme,1,Yok,NaN,NaN,NaN,Hayır,100 TL,Açık Balkon,1,NaN
613,31000TL,İstanbul,Ümraniye,Armağanevler Mahallesi,18615.0,15776848,28 Ağustos 2024,Kiralık,72 m²,2+1,4.Kat,Merkezi (Pay Ölçer),NaN,1,27 Ağustos 2024,Konut,Daire,85 m²,5-10,12,Boş,Evet,Genel Fiyat,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
808,25000TL,İstanbul,Ümraniye,İnkılap Mahallesi,20250.0,15791845,29 Ağustos 2024,Kiralık,80 m²,2+1,1.Kat,Kombi Doğalgaz,NaN,2,29 Ağustos 2024,Konut,Daire,90 m²,5-10,4,Boş,Hayır,Genel Fiyat,NaN,Boş,NaN,NaN,NaN,Yok,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [67]:
df.shape

(1500, 37)

In [68]:
import os
if not os.path.exists('data'):
    os.makedirs('data')

In [69]:
df.to_csv('data/emlakjet.csv', index=False)